# OpenRouter.ai demo

In [85]:
import requests
import json
import os
from dotenv import load_dotenv

In [86]:
# Load environment variables from .env file
load_dotenv()

# Check if API key is set
api_key = os.getenv('OPENROUTER_API_KEY_LLMA')
if not api_key:
    print("Error: OPENROUTER_API_KEY environment variable not set")
    exit(1)

In [87]:
response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        # "HTTP-Referer": "https://your-site-url.com",  # Optional
        # "X-Title": "Your Site Name",  # Optional
    },
    data=json.dumps({
        "model": "deepseek/deepseek-r1-distill-llama-70b:free",
        "messages": [
            {
                "role": "user",
                "content": "What is the meaning of life?"
            }
        ],
    })
)

In [88]:
response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        # "HTTP-Referer": "https://your-site-url.com",  # Optional
        # "X-Title": "Your Site Name",  # Optional
    },
    data=json.dumps({
        "model": "meta-llama/llama-3.2-3b-instruct:free",
        "messages": [
            {
                "role": "user",
                "content": "What is the meaning of life?"
            }
        ],
    })
)

In [89]:
# Check response deep-seek
if response.status_code == 200:
    try:
        data = response.json()
        print("Response:", data['choices'][0]['message']['content'])
    except (KeyError, ValueError) as e:
        print(f"Error parsing response: {e}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Response: The question of the meaning of life is one of the most profound and enduring questions humanity has ever posed. It is a question that has been debated and explored by philosophers, theologians, scientists, and thinkers of all kinds across various cultures and civilizations.

There is no one definitive answer to this question, as it is inherently subjective and can vary greatly depending on one's cultural background, personal beliefs, values, and experiences. That being said, here are some possible perspectives on the meaning of life:

1. **Biological perspective**: From a purely biological standpoint, the meaning of life can be seen as simply to survive and reproduce, ensuring the continuation of one's genetic lineage. This view is based on the idea that the universe has no inherent purpose or direction, and that life is just a random process of energy and matter evolving and adapting.
2. **Philosophical perspective**: Philosophers have long debated the nature of the meaning 

In [90]:
# Check response llma
if response.status_code == 200:
    try:
        data = response.json()
        print("Response:", data['choices'][0]['message']['content'])
    except (KeyError, ValueError) as e:
        print(f"Error parsing response: {e}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Response: The question of the meaning of life is one of the most profound and enduring questions humanity has ever posed. It is a question that has been debated and explored by philosophers, theologians, scientists, and thinkers of all kinds across various cultures and civilizations.

There is no one definitive answer to this question, as it is inherently subjective and can vary greatly depending on one's cultural background, personal beliefs, values, and experiences. That being said, here are some possible perspectives on the meaning of life:

1. **Biological perspective**: From a purely biological standpoint, the meaning of life can be seen as simply to survive and reproduce, ensuring the continuation of one's genetic lineage. This view is based on the idea that the universe has no inherent purpose or direction, and that life is just a random process of energy and matter evolving and adapting.
2. **Philosophical perspective**: Philosophers have long debated the nature of the meaning 

# Data vectorization

In [91]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [92]:
with open("shop_data.json", "r", encoding="utf-8") as f:
    products = json.load(f)

In [93]:
products = products['products']
products[0]

{'product_id': 'B1',
 'type': 'flower',
 'name': 'Red Rose Bouquet',
 'price': 40,
 'stock': 15,
 'available': True,
 'description': 'Assorted stems of seasonal roses.',
 'description_detail': 'A classic bouquet of vibrant red roses, perfect for romantic occasions.',
 'tip': 'Keep stems trimmed and water fresh daily.',
 'caution': 'Avoid direct sunlight for prolonged periods.',
 'image_url': ['demo_1', 'demo', 'demo_2', 'demo_3'],
 'care': {'sun': 'Partial'},
 'flower_details': {'occasion': ['Wedding'],
  'color': ['Pink Flowers'],
  'flower_type': 'Roses',
  'options': [{'name': 'original', 'price': 19.99, 'stems': 5},
   {'name': 'deluxe', 'price': 29.99, 'stems': 10},
   {'name': 'grand', 'price': 39.99, 'stems': 15}],
  'accessories': ['V1', 'V2']}}

In [94]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [95]:
def build_embedding_text(p: dict) -> str:
    # Lấy danh sách options nếu có
    options = p.get('flower_details', {}).get('options', [])
    
    if options:
        prices = [opt.get('price') for opt in options if isinstance(opt.get('price'), (int, float))]
        if prices:
            min_price = min(prices)
            max_price = max(prices)
            price_text = f"Price range: from {min_price} to {max_price} (optional)."
        else:
            price_text = ""
    else:
        price_text = ""

    # Tạo mô tả embedding
    return f"""
Name: {p['name']}
Description: {p.get('description', '')}
Detail Description: {p.get('description_detail', '')}
Type: {p.get('flower_details', {}).get('flower_type', '')}
Color: {', '.join(p.get('flower_details', {}).get('color', []))}
Occasion: {', '.join(p.get('flower_details', {}).get('occasion', []))}
Tip: {p.get('tip', '')}
Caution: {p.get('caution', '')}
Accessories: {', '.join(p.get('flower_details', {}).get('accessories', []))}
{price_text}
""".strip()


In [96]:
# Chọn text để encode (tùy chỉnh theo nhu cầu)
texts = [
    build_embedding_text(p) for p in products
]

In [97]:
texts[0]

'Name: Red Rose Bouquet\nDescription: Assorted stems of seasonal roses.\nDetail Description: A classic bouquet of vibrant red roses, perfect for romantic occasions.\nType: Roses\nColor: Pink Flowers\nOccasion: Wedding\nTip: Keep stems trimmed and water fresh daily.\nCaution: Avoid direct sunlight for prolonged periods.\nAccessories: V1, V2\nPrice range: from 19.99 to 39.99 (optional).'

In [98]:
texts

['Name: Red Rose Bouquet\nDescription: Assorted stems of seasonal roses.\nDetail Description: A classic bouquet of vibrant red roses, perfect for romantic occasions.\nType: Roses\nColor: Pink Flowers\nOccasion: Wedding\nTip: Keep stems trimmed and water fresh daily.\nCaution: Avoid direct sunlight for prolonged periods.\nAccessories: V1, V2\nPrice range: from 19.99 to 39.99 (optional).',
 "Name: Farmer's Choice Peonies\nDescription: Assorted stems of seasonal peonies.\nDetail Description: A rustic mix of peonies, hand-picked for freshness.\nType: Peonies\nColor: White Flowers\nOccasion: New Baby, Thank you\nTip: Place in a cool area away from heat sources.\nCaution: Petals are delicate; handle with care.\nAccessories: V1, V2\nPrice range: from 19.99 to 39.99 (optional).",
 'Name: Peony Bouquet\nDescription: Beautiful peony flowers.\nDetail Description: Lush peonies symbolizing prosperity and love.\nType: Peonies\nColor: Pink Flowers\nOccasion: Wedding, Graduation\nTip: Change water eve

In [99]:
# Tạo embedding
embeddings = model.encode(texts, convert_to_numpy=True)

In [100]:
# Lưu vào FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [101]:
# Lưu index và products
faiss.write_index(index, "faiss_index.index")
with open("product_texts.json", "w", encoding="utf-8") as f:
    json.dump(texts, f, ensure_ascii=False)
with open("product_data.json", "w", encoding="utf-8") as f:
    json.dump(products, f, ensure_ascii=False)

# Load the model and JSON data

In [103]:
# Load FAISS + dữ liệu
model = SentenceTransformer('all-MiniLM-L6-v2')
faiss_index = faiss.read_index("faiss_index.index")

In [104]:
with open("product_data.json", "r", encoding="utf-8") as f:
    product_data = json.load(f)
with open("product_texts.json", "r", encoding="utf-8") as f:
    product_texts = json.load(f)

def semantic_search(query, top_k=3):
    embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = faiss_index.search(embedding, top_k)
    results = [product_data[i] for i in indices[0]]
    return results

In [105]:
question = "Please give me the price of the rosa"

In [ ]:
semantic_search(question, 1)

[{'product_id': 'B1',
  'type': 'flower',
  'name': 'Red Rose Bouquet',
  'price': 40,
  'stock': 15,
  'available': True,
  'description': 'Assorted stems of seasonal roses.',
  'description_detail': 'A classic bouquet of vibrant red roses, perfect for romantic occasions.',
  'tip': 'Keep stems trimmed and water fresh daily.',
  'caution': 'Avoid direct sunlight for prolonged periods.',
  'image_url': ['demo_1', 'demo', 'demo_2', 'demo_3'],
  'care': {'sun': 'Partial'},
  'flower_details': {'occasion': ['Wedding'],
   'color': ['Pink Flowers'],
   'flower_type': 'Roses',
   'options': [{'name': 'original', 'price': 19.99, 'stems': 5},
    {'name': 'deluxe', 'price': 29.99, 'stems': 10},
    {'name': 'grand', 'price': 39.99, 'stems': 15}],
   'accessories': ['V1', 'V2']}}]

# Combine with the user request

In [151]:
def user_prompt_handler(user_message: str, k: int = 1) -> str:
    
    # Thay vì dùng regex, dùng semantic search
    retrieved_products = semantic_search(user_message, k)
    context_text = "\n".join([json.dumps(p, ensure_ascii=False) for p in retrieved_products])

    prompt_with_context = f"""
You are an assistant that answers user questions based strictly on the provided data.

[Shop Data]:
{context_text}

[User Question]:
"{user_message}"

[Instructions]:
1. Only use facts and wording that appear exactly in the [Shop Data].
2. Do not invent or assume anything not shown in the data.
3. Do not add extra punctuation, currency symbols (like "$"), or any formatting not present in the original data.
4. If the information is not found in the [Shop Data], respond exactly with: Data not found.
5. Rephrase the response naturally, as if you are talking to a customer.
6. List product names clearly, using '-' if there are multiple items.
7. Do NOT use JSON or code format in your response. 
    """

    
    return prompt_with_context

In [152]:
user_prompt_handler("Please provide me the price of the flower", 2)

'\nYou are an assistant that answers user questions based strictly on the provided data.\n\n[Shop Data]:\n{"product_id": "B1", "type": "flower", "name": "Red Rose Bouquet", "price": 40, "stock": 15, "available": true, "description": "Assorted stems of seasonal roses.", "description_detail": "A classic bouquet of vibrant red roses, perfect for romantic occasions.", "tip": "Keep stems trimmed and water fresh daily.", "caution": "Avoid direct sunlight for prolonged periods.", "image_url": ["demo_1", "demo", "demo_2", "demo_3"], "care": {"sun": "Partial"}, "flower_details": {"occasion": ["Wedding"], "color": ["Pink Flowers"], "flower_type": "Roses", "options": [{"name": "original", "price": 19.99, "stems": 5}, {"name": "deluxe", "price": 29.99, "stems": 10}, {"name": "grand", "price": 39.99, "stems": 15}], "accessories": ["V1", "V2"]}}\n{"product_id": "B3", "type": "flower", "name": "Peony Bouquet", "price": 40, "stock": 18, "available": true, "description": "Beautiful peony flowers.", "de

# Integrate with the Chatbot

In [153]:
import requests
import json
import os
from dotenv import load_dotenv

In [154]:
# Load environment variables from .env file
load_dotenv()

# Check if API key is set
api_key = os.getenv('OPENROUTER_API_KEY_LLMA')
if not api_key:
    print("Error: OPENROUTER_API_KEY environment variable not set")
    exit(1)

In [155]:
def llm_response(user_message: str, k: int = 3):

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            # "HTTP-Referer": "https://your-site-url.com",  # Optional
            # "X-Title": "Your Site Name",  # Optional
        },
        data=json.dumps({
            "model": "deepseek/deepseek-r1-distill-llama-70b:free",
            "messages": [
                {
                    "role": "user",
                    "content": f"{user_prompt_handler(user_message, k)}"
                }
            ],
        })
    )
    return response

In [159]:
user_message_price = "Please give me the price of the Rose with grand version"

In [160]:
user_message_list = "Please give me some vase product"

In [163]:
user_message_context = "Help me to find flower for romantic scenario"

In [161]:
response = llm_response(user_message_price)

# Check response deep-seek
if response.status_code == 200:
    try:
        data = response.json()
        print("Response:", data['choices'][0]['message']['content'])
    except (KeyError, ValueError) as e:
        print(f"Error parsing response: {e}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Response: The price of the Red Rose Bouquet in the grand version is 39.99.


In [162]:
response = llm_response(user_message_list)

# Check response deep-seek
if response.status_code == 200:
    try:
        data = response.json()
        print("Response:", data['choices'][0]['message']['content'])
    except (KeyError, ValueError) as e:
        print(f"Error parsing response: {e}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Response: Here are some vase products available:

- Montecito Vase - A beautiful vase for your flowers. Elegant glass vase with a sleek design. Price: 19, Stock: 12
- Montecitoss Vase - A beautiful vase for your flowers. Stylish vase with a unique finish. Price: 25, Stock: 12

Let me know if you need more details!


In [164]:
response = llm_response(user_message_context)

# Check response deep-seek
if response.status_code == 200:
    try:
        data = response.json()
        print("Response:", data['choices'][0]['message']['content'])
    except (KeyError, ValueError) as e:
        print(f"Error parsing response: {e}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Response: Here are some flower suggestions suitable for romantic occasions:

- Red Rose Bouquet (Product ID: B1) - Price: 40, Stock: 15, Description: A classic bouquet of vibrant red roses, perfect for romantic occasions, Flower Details: Color - Pink Flowers, Flower Type - Roses
- Peony Bouquet (Product ID: B3) - Price: 40, Stock: 18, Description: Lush peonies symbolizing prosperity and love, Flower Details: Color - Pink Flowers, Flower Type - Peonies

These options are available and perfect for romantic scenarios.


In [165]:
print("Here are the vase products available:\n\n- Montecito Vase  \n  Price: 19  \n  Stock: 12  \n  Description: Elegant glass vase with a sleek design.  \n\n- Montecitoss Vase  \n  Price: 25  \n  Stock: 12  \n  Description: Stylish vase with a unique finish.  \n\nLet me know if you'd like more details!")

Here are the vase products available:

- Montecito Vase  
  Price: 19  
  Stock: 12  
  Description: Elegant glass vase with a sleek design.  

- Montecitoss Vase  
  Price: 25  
  Stock: 12  
  Description: Stylish vase with a unique finish.  

Let me know if you'd like more details!
